<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/C(deeplog_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

alternative files :


```

# download datasets
!wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_train'
!wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_normal'
!wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_abnormal'
     
```
be careful these files are logs , not csv


In [1]:
# Mount Google Drive to upload datasets (csv files)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:

import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np
import pandas as pd

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [64]:
names = ['hdfs_sequence_normal','hdfs_sequence_abnormal']
templates = set()

for name in names:
  with open('/content/drive/MyDrive/HDFS/structured_hdfs/' + name, 'r') as f:
          for row in f:
            for temp in row.split():
              templates.add(temp)

print(sorted(templates))
print(len(templates))

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '5', '6', '7', '8', '9']
48


In [38]:
hdfs = pd.read_csv('/content/drive/MyDrive/HDFS/structured_hdfs/hdfs.csv')
hdfs.head()

,Unnamed: 0.1,Unnamed: 0,BlockId,EventSequence,Label
0,0,0,blk_-1608999687919862906,"[0, 1, 0, 0, 2, 2, 3, 3, 2, 3, 4, 4, 4, 5, 0, ...",Normal
1,1,1,blk_7503483334202473044,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 9, 12,...",Normal
2,2,2,blk_-3544583377289625738,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 9, 4, 4, 4, 9, ...",Anomaly
3,3,3,blk_-9073992586687739851,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12...",Normal
4,4,4,blk_7854771516489510256,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12...",Normal


In [58]:
def split_data(hdfs,train_ratio = 0.7):

  # convert hdfs['EventSequence'] from str into lists
  #hdfs['EventSequence'] = hdfs['EventSequence'].apply(ast.literal_eval)

  # Splitting the dataset into train and test
  normal_data_df = hdfs[hdfs['Label'] == 'Normal']

  train_ratio = 0.7  # 80% for training
  train_normal_len = int(len(normal_data_df) * train_ratio)

  hdfs_train = normal_data_df[:train_normal_len]
  hdfs_test_normal = normal_data_df[train_normal_len:]

  hdfs_test_abnormal = hdfs[hdfs['Label'] != 'Normal']

  # Print the lengths of train and test datasets
  print("Train Dataset Length:", len(hdfs_train))
  print("Normal Test Dataset Length:", len(hdfs_test_normal))
  print("Abnormal Test Dataset Length:", len(hdfs_test_abnormal))

  return hdfs_train, hdfs_test_normal, hdfs_test_abnormal


In [59]:
hdfs_train, hdfs_test_normal, hdfs_test_abnormal = split_data(hdfs, train_ratio = 0.7)

Train Dataset Length: 390756
Normal Test Dataset Length: 167467
Abnormal Test Dataset Length: 16838


In [40]:
# convert hdfs['EventSequence'] from str into lists
import ast
hdfs['EventSequence'] = hdfs['EventSequence'].apply(ast.literal_eval)

In [42]:
# Splitting the dataset into train and test
normal_data_df = hdfs[hdfs['Label'] == 'Normal']

train_ratio = 0.7  # 80% for training
train_normal_len = int(len(normal_data_df) * train_ratio)

hdfs_train = normal_data_df[:train_normal_len]
hdfs_test_normal = normal_data_df[train_normal_len:]

hdfs_test_abnormal = hdfs[hdfs['Label'] != 'Normal']

# Print the lengths of train and test datasets
print("Train Dataset Length:", len(hdfs_train))
print("Normal Test Dataset Length:", len(hdfs_test_normal))
print("Abnormal Test Dataset Length:", len(hdfs_test_abnormal))

Train Dataset Length: 390756
Normal Test Dataset Length: 167467
Abnormal Test Dataset Length: 16838


In [43]:
hdfs_test_abnormal

,Unnamed: 0.1,Unnamed: 0,BlockId,EventSequence,Label
2,2,2,blk_-3544583377289625738,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 9, 4, 4, 4, 9, ...",Anomaly
15,15,15,blk_-8531310335568756456,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12...",Anomaly
23,23,23,blk_3947106522258141922,"[0, 0, 1, 0, 2, 3, 2, 3, 4, 4, 2, 3, 4, 12, 12...",Anomaly
64,64,64,blk_7956543127401791181,"[0, 0, 1, 0, 3, 2, 3, 4, 4, 2, 3, 2, 4, 12, 12...",Anomaly
91,91,91,blk_-3102267849859399193,"[0, 0, 1, 13, 2, 14, 18, 19]",Anomaly
...,...,...,...,...,...
574651,574651,574651,blk_-8186558713608306359,"[0, 1, 0, 0, 4, 17, 2, 3, 17, 2, 3, 17, 2, 3, ...",Anomaly
574900,574900,574900,blk_-1683688019270077407,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 20, 31...",Anomaly
574924,574924,574924,blk_3773788001864047190,"[0, 0, 0, 1, 17, 2, 3, 17, 2, 3, 17, 2, 3, 4, ...",Anomaly
575002,575002,575002,blk_1986112558741936483,"[17, 0, 2, 3, 17, 0, 2, 3, 17, 0, 2, 3, 1, 4, ...",Anomaly


In [54]:
with open('hdfs_train', 'w') as log_file:
  for _ , row in hdfs_train.iterrows():
      log_entry  = ' '.join(str(token) for token in row['EventSequence'])
      log_file.write(log_entry + '\n')

with open('hdfs_test_normal', 'w') as log_file:
  for _ , row in hdfs_test_normal.iterrows():
      log_entry  = ' '.join(str(token) for token in row['EventSequence'])
      log_file.write(log_entry + '\n')

with open('hdfs_test_abnormal', 'w') as log_file:
  for _ , row in hdfs_test_abnormal.iterrows():
      log_entry  = ' '.join(str(token) for token in row['EventSequence'])
      log_file.write(log_entry + '\n')

In [ ]:
def write_dataframe_to_log(df, filename):
    with open(filename, 'w') as log_file:
        for _, row in df.iterrows():
            log_entry = ' '.join(str(token) for token in row['EventSequence'])
            log_file.write(log_entry + '\n')

# Example usage with three DataFrames hdfs_train, hdfs_test_normal, and hdfs_test_abnormal
write_dataframe_to_log(hdfs_train, 'hdfs_train.log')
write_dataframe_to_log(hdfs_test_normal, 'hdfs_test_normal.log')
write_dataframe_to_log(hdfs_test_abnormal, 'hdfs_test_abnormal.log')


In [60]:
import pandas as pd
import ast

def split_data_and_write_logs(hdfs, train_ratio=0.7):
    # convert hdfs['EventSequence'] from str into lists
    hdfs['EventSequence'] = hdfs['EventSequence'].apply(ast.literal_eval)

    # Splitting the dataset into train and test
    normal_data_df = hdfs[hdfs['Label'] == 'Normal']

    train_ratio = train_ratio  # You can adjust the train_ratio parameter here
    train_normal_len = int(len(normal_data_df) * train_ratio)

    hdfs_train = normal_data_df[:train_normal_len]
    hdfs_test_normal = normal_data_df[train_normal_len:]

    hdfs_test_abnormal = hdfs[hdfs['Label'] != 'Normal']

    # Print the lengths of train and test datasets
    print("Train Dataset Length:", len(hdfs_train))
    print("Normal Test Dataset Length:", len(hdfs_test_normal))
    print("Abnormal Test Dataset Length:", len(hdfs_test_abnormal))

    # Write only the 'EventSequence' column to log files
    hdfs_train['EventSequence'].to_csv('hdfs_train.log', sep=' ', index=False, header=False)
    hdfs_test_normal['EventSequence'].to_csv('hdfs_test_normal.log', sep=' ', index=False, header=False)
    hdfs_test_abnormal['EventSequence'].to_csv('hdfs_test_abnormal.log', sep=' ', index=False, header=False)

    return hdfs_train, hdfs_test_normal, hdfs_test_abnormal

hdfs = pd.read_csv('/content/drive/MyDrive/HDFS/structured_hdfs/hdfs.csv')
hdfs_train, hdfs_test_normal, hdfs_test_abnormal = split_data_and_write_logs(hdfs, train_ratio=0.8)


Train Dataset Length: 446578
Normal Test Dataset Length: 111645
Abnormal Test Dataset Length: 16838


In [61]:
import pandas as pd
import ast

def split_data_and_write_logs(hdfs, train_ratio=0.7):
    # convert hdfs['EventSequence'] from str into lists
    hdfs['EventSequence'] = hdfs['EventSequence'].apply(ast.literal_eval)

    # Splitting the dataset into train and test
    normal_data_df = hdfs[hdfs['Label'] == 'Normal']

    train_ratio = train_ratio  # You can adjust the train_ratio parameter here
    train_normal_len = int(len(normal_data_df) * train_ratio)

    hdfs_train = normal_data_df[:train_normal_len]
    hdfs_test_normal = normal_data_df[train_normal_len:]

    hdfs_test_abnormal = hdfs[hdfs['Label'] != 'Normal']

    # Print the lengths of train and test datasets
    print("Train Dataset Length:", len(hdfs_train))
    print("Normal Test Dataset Length:", len(hdfs_test_normal))
    print("Abnormal Test Dataset Length:", len(hdfs_test_abnormal))

    # Write only the 'EventSequence' column to log files
    with open('hdfs_train.log', 'w') as log_file:
        for _, row in hdfs_train.iterrows():
            log_entry = ' '.join(str(token) for token in row['EventSequence'])
            log_file.write(log_entry + '\n')

    with open('hdfs_test_normal.log', 'w') as log_file:
        for _, row in hdfs_test_normal.iterrows():
            log_entry = ' '.join(str(token) for token in row['EventSequence'])
            log_file.write(log_entry + '\n')

    with open('hdfs_test_abnormal.log', 'w') as log_file:
        for _, row in hdfs_test_abnormal.iterrows():
            log_entry = ' '.join(str(token) for token in row['EventSequence'])
            log_file.write(log_entry + '\n')

    return

hdfs = pd.read_csv('/content/drive/MyDrive/HDFS/structured_hdfs/hdfs.csv')
split_data_and_write_logs(hdfs, train_ratio=0.8)


Train Dataset Length: 446578
Normal Test Dataset Length: 111645
Abnormal Test Dataset Length: 16838
